# What does my target market want

In [ ]:
df_listings = pd.read_csv('./listings.csv')
df_listings.info()

In [ ]:
#Take a look at the amenities of the current listings
df_amenities = df_listings['amenities'].value_counts().reset_index()
df_amenities.head()

In [ ]:
# The items are grouped together if the listing contains more than one amenity. 
# We need to clean this up by plsitting the list items into rows
df_listings['amenities'] = df_listings['amenities'].str.replace('"', '')
df_listings['amenities'] = df_listings['amenities'].str.replace('{', '')
df_listings['amenities'] = df_listings['amenities'].str.replace('}', '')

In [ ]:
# Function to get total count of a column with lists

def total_count(df, col):
    
    count_rows = df.shape[0]
    
    new_df = df[col].value_counts().reset_index()
    new_df.rename(columns={'index': col, col: 'count'}, inplace=True)
    new_df[col] = new_df[col].str.split(",")
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.groupby([col]).sum()['count'].reset_index()
    new_df = new_df.sort_values('count', ascending=False)
    new_df['perc'] = new_df['count']/count_rows
    
    return new_df

In [ ]:
df_total = total_count(df_listings, 'amenities')

In [ ]:
# My new metric

df_listings['EstimatedIncome'] = df_listings['price'] * (365-df_listings['availability_365'])
df_listings['EstimatedIncome'] 

In [ ]:
#label = df_listings["reviews_per_month"]
label = df_listings["EstimatedIncome"]
fig, ax = plt.subplots(1, 1, figsize = (9,4.5))

# Plot the histogram   
ax.hist(label, bins=100)
ax.set_ylabel('Frequency')

# Add lines for the mean, median, and mode
ax.axvline(label.mean(), color='magenta', linestyle='dashed', linewidth=2)
ax.axvline(label.median(), color='cyan', linestyle='dashed', linewidth=2)

# Add a title to the Figure
fig.suptitle('Estimated annual income')


In [ ]:
df_listings["EstimatedIncome"].describe()

In [ ]:
#Lets check what the top 75% of people are doing/ have

df_Top75 = df_listings[(df_listings["EstimatedIncome"] >= 23622.500000)]
df_Top75["amenities"]

In [ ]:
df_theTop = total_count(df_Top75, "amenities")

In [ ]:
df_combined = pd.merge(df_total, df_theTop, on='amenities', how = 'inner')
df_combined['diff'] = df_combined['perc_y'] - df_combined['perc_x']
df_combined.sort_values('diff', ascending =False)

In [ ]:
x = np.arange(len(df_combined['amenities']))
width = 0.3

labels = df_combined['amenities']
y1 = df_combined['perc_x']
y2 = df_combined['perc_y']
y3 = df_combined['diff']


fig, ax = plt.subplots()
#bar1 = ax.bar(x - width/2, y1, width, label='Overall')
#bar2 = ax.bar(x + width/2, y2, width, label='The top')
bar3 = ax.bar(x + width/2, y3, width, label='The top')

ax.set_ylabel('importance')
ax.set_title('The title')
ax.set_xticks(x, labels)
ax.legend()

#ax.bar_label(bar1)
#ax.bar_label(bar2)
#ax.bar_label(bar3)